In [ ]:
import argparse
import datetime
import glob
import os.path
import pandas as pd
import backtrader as bt
import backtrader.analyzers as btanal
import numpy as np
from scipy.stats import linregress

def momentum_func(self, the_array):
    r = np.log(the_array)
    slope, _, rvalue, _, _ = linregress(np.arange(len(r)), r)
    annualized = (1 + slope) ** 252
    return annualized * (rvalue ** 2)

class Momentum(bt.ind.OperationN):
    lines = ('trend',)
    #params = dict(period=50)
    func = momentum_func

class St(bt.Strategy):
    params = dict(
        selcperc=0.10,  # percentage of stocks to select from the universe
        rperiod=1,  # period for the returns calculation, default 1 period
        vperiod=30,  # lookback period for volatility - default 36 periods
        mperiod=30,  # lookback period for momentum - default 90 periods
        momentum=Momentum, # parametrize the momentum and its period
        reserve=0.05,  # 5% reserve capital
        monthdays=[1],
        monthcarry=True,
        when=bt.timer.SESSION_START,
        timer=True
    )

    def log(self, arg):
        print('{} {}'.format(self.datetime.date(), arg))

    def __init__(self):
        # calculate 1st the amount of stocks that will be selected
        self.selnum = int(len(self.datas) * self.p.selcperc)

        # allocation perc per stock
        # reserve kept to make sure orders are not rejected due to
        # margin. Prices are calculated when known (close), but orders can only
        # be executed next day (opening price). Price can gap upwards
        self.perctarget = (1.0 - self.p.reserve) / self.selnum
        
        self.add_timer(
            when=self.p.when,
            monthdays=self.p.monthdays,
            monthcarry=self.p.monthcarry
        )
       
        # returns, volatilities and momentums
        rs = [bt.ind.PctChange(d, period=self.p.rperiod) for d in self.datas]
        vs = [bt.ind.StdDev(ret, period=self.p.vperiod) for ret in rs]
        #ms = [bt.ind.ROC(d, period=self.p.mperiod) for d in self.datas]
        ms = [self.p.momentum(d, period=self.p.mperiod) for d in self.datas]

        # simple rank formula: (momentum * net payout) / volatility
        # the highest ranked: low vol, large momentum, large payout
        self.ranks = {d: m / v for d, v, m in zip(self.datas, vs, ms)}
        #TODO: does it perform better without the volatility?
  
    def prenext(self):
        # call next() even when data is not available for all tickers
        self.next()     

    def notify_timer(self, timer, when, *args, **kwargs):
        if self._getminperstatus() < 0:
            self.rebalance()
    
    def next(self):
        pass
    def rebalance(self):
        # sort data and current rank
        ranks = sorted(
            self.ranks.items(),  # get the (d, rank), pair
            key=lambda x: x[1][0],  # use rank (elem 1) and current time "0"
            reverse=True,  # highest ranked 1st ... please
        )
        
        # put top ranked in dict with data as key to test for presence
        rtop = dict(ranks[:self.selnum])

        # For logging purposes of stocks leaving the portfolio
        rbot = dict(ranks[self.selnum:])

        # prepare quick lookup list of stocks currently holding a position
        posdata = [d for d, pos in self.getpositions().items() if pos]

        # remove those no longer top ranked
        # do this first to issue sell orders and free cash
        for d in (d for d in posdata if d not in rtop):
            self.log('Leave {} - Rank {:.2f}'.format(d._name, rbot[d][0]))
            self.order_target_percent(d, target=0.0)

        # rebalance those already top ranked and still there
        for d in (d for d in posdata if d in rtop):
            self.log('Rebal {} - Rank {:.2f}'.format(d._name, rtop[d][0]))
            self.order_target_percent(d, target=self.perctarget)
            del rtop[d]  # remove it, to simplify next iteration

        # issue a target order for the newly top ranked stocks
        # do this last, as this will generate buy orders consuming cash
        for d in rtop:
            self.log('Enter {} - Rank {:.2f}'.format(d._name, rtop[d][0]))
            self.order_target_percent(d, target=self.perctarget)


def run(args=None):
    cerebro = bt.Cerebro()


    # Parse from/to-date
    fromdate = datetime.datetime(2013, 3, 1)
    todate = datetime.datetime(2018, 6, 18)

    # add all the data files available in the directory datadir
    for fname in glob.glob(os.path.join(r'C:/Users/MMD/PycharmProjects/Trading/Data Mining/Data/data', '*')):
        df = pd.read_csv(fname, index_col='date', parse_dates=True)
        #if (df.index == fromdate).any():
        #cerebro.adddata(bt.feeds.PandasData(dataname=df, fromdate=fromdate, todate=todate, plot=False))
        if len(df)>100:
            cerebro.adddata(bt.feeds.PandasData(dataname=df))

    # add strategy
    cerebro.addstrategy(St)

    # set the cash
    cerebro.broker.setcash(10000)
    cerebro.broker.set_coc(True)
    cerebro.broker.setcommission(commission=0.005)
    
    #Analysers such as Sharpe Ratio
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.Returns)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    cerebro.addanalyzer(btanal.PyFolio)                # Needed to use PyFolio
    cerebro.addanalyzer(btanal.TradeAnalyzer)          # Analyzes individual trades
    cerebro.addanalyzer(btanal.SharpeRatio_A)          # Gets the annualized Sharpe ratio
    cerebro.addanalyzer(btanal.AnnualReturn)          # Annualized returns (does not work?)

    
    results = cerebro.run(stdstats=True, tradehistory=True)
    cerebro.plot(style = 'candle')
    print(f"Sharpe: {results[0].analyzers.sharperatio.get_analysis()['sharperatio']:.3f}")
    print(f"Norm. Annual Return: {results[0].analyzers.returns.get_analysis()['rnorm100']:.2f}%")
    print(f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")

    # Basic performance evaluation ... final value ... minus starting cash
    pnl = cerebro.broker.get_value() - 10000
    print('Profit ... or Loss: {:.2f}'.format(pnl))

if __name__ == '__main__':
    run()